In [136]:
import tensorflow as tf
import pandas as pd

In [137]:
df = pd.read_csv('../data/restaurant_reviews.csv',
                )


# ds.drop(['Restaurant','Reviewer','Time','Pictures','Metadata','7514'], inplace=True ,axis=1)
df.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5.0,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5.0,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5.0,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5.0,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5.0,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [138]:
#convert into tf.dataset 
df['Rating']=df['Rating'].astype('float32')
# print(df['Review'].values)
dataset = tf.data.Dataset.from_tensor_slices((df['Review'].astype(str).values, df['Rating'].values))

In [139]:
#splliting dataset
SIZE = 0.8
num_elements=len(df['Review'].astype(str).values)
train_size = int(0.8 * num_elements)

# Define the size of the testing set
test_size = num_elements - train_size

train_dataset = dataset.skip(test_size)
test_dataset = dataset.take(test_size)

In [140]:
BUFFER_SIZE=1000
BATCH_SIZE=64
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [141]:
VOCAB_SIZE = 150000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
df_tensor = tf.convert_to_tensor(df["Review"].astype(str), dtype=tf.string)
encoder.adapt(df_tensor)

In [143]:
embed = tf.keras.layers.Embedding(
    150000,
    128,
)

model = tf.keras.Sequential([
    embed,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 1: unexpected end of data

In [123]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=20,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
125/125 [==============================] - 28s 138ms/step - loss: nan - accuracy: 0.0261 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
125/125 [==============================] - 17s 140ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/10
125/125 [==============================] - 19s 155ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
125/125 [==============================] - 14s 113ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
125/125 [==============================] - 17s 140ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
125/125 [==============================] - 19s 154ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
125/125 [==============================] - 18s 144ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - 

KeyboardInterrupt: 